In [11]:
# Import important libraries

import numpy as np
import sympy as sp
import scipy.integrate as integrate

In [12]:
# Block for all inputs

a1 = 24.001
a2 = 24
a3 = 12
a = np.array([a1, a2, a3])

eps11 = 0.001
eps22 = 0.001
eps33 = 0.001
eps12 = 0
eps13 = 0
eps23 = 0
app_strain = np.array([[eps11,eps12,eps13],[eps12,eps22,eps23],[eps13,eps23,eps33]])

nu = 0.3
E = 210


der_epsi = 0.001

delta = np.identity(3)
lambda_ = 1


In [13]:
# Solving the cubis equation of lambda (page 86 of Mura's book, equation 11.37)
def get_lambda(x,a):
    global lambda_
    sym = sp.Symbol('lambda_')
    equation = (x[0]**2)/(a[0]**2 + sym) + (x[1]**2)/(a[1]**2 + sym) + (x[2]**2)/(a[2]**2 + sym) - 1
    sol =  sp.solve(equation, sym)
    #print("Solution of Lambda: ",sol)

    # Convert sol to complex number
    sol = [complex(i) for i in sol]
    #print(sol)

    # Select the real part of the solution
    sol = [i.real for i in sol]
    #print(sol)
    
    sol = max(sol)
    if sol > 0:
        lambda_ = sol
    else:
        lambda_ = 0
    return lambda_
    #print("lambda = ",lambda_)

In [14]:
x = np.array([25,0,0])
a = np.array([a1,a2,a3])
lambda_ = get_lambda(x,a)
print("lambda = ",lambda_)

lambda =  48.951999


In [15]:
# Defining elliptic integrals and solving them. (page 85 of Mura's book, equation 11.36) lambda=0.. verified for lambda = 0
def I(x,a):
    #lambda_ = get_lambda(x,a)
    global lambda_
    def integrand(s):
        return 1/np.sqrt((a[0]**2 + s)*(a[1]**2 + s)*(a[2]**2 + s))
    inte = integrate.quad(integrand, lambda_, np.inf)[0]
    return inte*np.pi*2*a[0]*a[1]*a[2]

def Ii(x,a,i):
    #lambda_ = get_lambda(x,a)
    global lambda_
    def integrand(s):
        return (1/(np.sqrt((a[0]**2 + s)*(a[1]**2 + s)*(a[2]**2 + s))*(a[i]**2 + s)))
    inte = integrate.quad(integrand, lambda_, np.inf)[0]
    return inte*np.pi*2*a[0]*a[1]*a[2]

def Iij(x,a,i,j):
    #lambda_ = get_lambda(x,a)
    global lambda_
    def integrand(s):
        return (1/(np.sqrt((a[0]**2 + s)*(a[1]**2 + s)*(a[2]**2 + s))*(a[i]**2 + s)*(a[j]**2 + s)))
    inte = integrate.quad(integrand, lambda_, np.inf)[0]
    return inte*np.pi*2*a[0]*a[1]*a[2]

In [16]:
for i in range(3):
    for j in range(3):
        print("I",i,j," = ",Iij(x,a,i,j))

I 0 0  =  0.002581469717314887
I 0 1  =  0.002581614194079541
I 0 2  =  0.005705460002393
I 1 0  =  0.002581614194079541
I 1 1  =  0.0025817586818391246
I 1 2  =  0.005705807729248813
I 2 0  =  0.005705460002392998
I 2 1  =  0.00570580772924881
I 2 2  =  0.013481431476441383


In [17]:
for i in range(3):
    print("I",i," = ",Ii(x,a,i))

I 0  =  2.5134841909743573
I 1  =  2.5136078408801596
I 2  =  4.978522677229332


In [18]:
# Compute Sijkl (page 88 of Mura's book, equation 11.42)

def get_Sijkl(x,a):
    Sijkl = np.zeros((3,3,3,3))
    for i in range(3):
        for j in range(3):
            for k in range(3):
                for l in range(3):
                    t1 = delta[i,j]*delta[k,l]*(2*nu*Ii(x,a,i) - Ii(x,a,k) + (a[i]**2)*Iij(x,a,k,i))
                    t2 = (delta[i,k]*delta[j,l] + delta[i,l]*delta[j,k])*((a[i]**2)*Iij(x,a,i,j) - Ii(x,a,j) + (1-nu)*(Ii(x,a,k) + Ii(x,a,l)))
                    Sijkl[i,j,k,l] = (t1+t2)/(8*np.pi*(1-nu))
                    print("S",i+1,j+1,k+1,l+1," = ",Sijkl[i,j,k,l])
    return Sijkl

In [19]:
# Compute derivatives of elliptic integrals (page 86 of Mura's book, equation 11.39)

#Compute derivative of Ii wrt to j direction
def Ii_j(x,a,i,j):
    xp = x + der_epsi*delta[j]
    xm = x - der_epsi*delta[j]
    return (Ii(xp,a,i) - Ii(xm,a,i))/(2*der_epsi)

# Compute derivative of Iij wrt to k direction
def Iij_k(x,a,i,j,k):
    xp = x + der_epsi*delta[k]
    xm = x - der_epsi*delta[k]
    return (Iij(xp,a,i,j) - Iij(xm,a,i,j))/(2*der_epsi)

# Compute double partial derivative of Iij wrt to k and l direction
def Iij_kl(x,a,i,j,k,l):
    xp_yp = x + der_epsi*delta[k] + der_epsi*delta[l]
    xm_yp = x - der_epsi*delta[k] + der_epsi*delta[l]
    xp_ym = x + der_epsi*delta[k] - der_epsi*delta[l]
    xm_ym = x - der_epsi*delta[k] - der_epsi*delta[l]
    return (Iij(xp_yp,a,i,j) - Iij(xm_yp,a,i,j) - Iij(xp_ym,a,i,j) + Iij(xm_ym,a,i,j))/(4*der_epsi**2)

# Compute derivative of Ii wrt to j and k direction
def Ii_jk(x,a,i,j,k):
    xp_yp = x + der_epsi*delta[j] + der_epsi*delta[k]
    xm_ym = x - der_epsi*delta[j] - der_epsi*delta[k]
    xp_ym = x + der_epsi*delta[j] - der_epsi*delta[k]
    xm_yp = x - der_epsi*delta[j] + der_epsi*delta[k]
    return (Ii(xp_yp,a,i) - Ii(xm_ym,a,i) - Ii(xp_ym,a,i) + Ii(xm_yp,a,i))/(4*der_epsi**2)

In [20]:
for i in range(3):
    for j in range(3):
        print("I",i+1,"_",j+1," = ",Ii_j(x,a,i,j))

I 1 _ 1  =  0.0
I 1 _ 2  =  0.0
I 1 _ 3  =  0.0
I 2 _ 1  =  0.0
I 2 _ 2  =  0.0
I 2 _ 3  =  0.0
I 3 _ 1  =  0.0
I 3 _ 2  =  0.0
I 3 _ 3  =  0.0


In [21]:
for i in range(3):
    for j in range(3):
        for k in range(3):
            for l in range(3):
                print("I",i+1,j+1,"_",k+1,l+1," = ",Iij_kl(x,a,i,j,k,l))

I 1 1 _ 1 1  =  0.0
I 1 1 _ 1 2  =  0.0
I 1 1 _ 1 3  =  0.0
I 1 1 _ 2 1  =  0.0
I 1 1 _ 2 2  =  0.0
I 1 1 _ 2 3  =  0.0
I 1 1 _ 3 1  =  0.0
I 1 1 _ 3 2  =  0.0
I 1 1 _ 3 3  =  0.0
I 1 2 _ 1 1  =  0.0
I 1 2 _ 1 2  =  0.0
I 1 2 _ 1 3  =  0.0
I 1 2 _ 2 1  =  0.0
I 1 2 _ 2 2  =  0.0
I 1 2 _ 2 3  =  0.0
I 1 2 _ 3 1  =  0.0
I 1 2 _ 3 2  =  0.0
I 1 2 _ 3 3  =  0.0
I 1 3 _ 1 1  =  0.0
I 1 3 _ 1 2  =  0.0
I 1 3 _ 1 3  =  0.0
I 1 3 _ 2 1  =  0.0
I 1 3 _ 2 2  =  0.0
I 1 3 _ 2 3  =  0.0
I 1 3 _ 3 1  =  0.0
I 1 3 _ 3 2  =  0.0
I 1 3 _ 3 3  =  0.0
I 2 1 _ 1 1  =  0.0
I 2 1 _ 1 2  =  0.0
I 2 1 _ 1 3  =  0.0
I 2 1 _ 2 1  =  0.0
I 2 1 _ 2 2  =  0.0
I 2 1 _ 2 3  =  0.0
I 2 1 _ 3 1  =  0.0
I 2 1 _ 3 2  =  0.0
I 2 1 _ 3 3  =  0.0
I 2 2 _ 1 1  =  0.0
I 2 2 _ 1 2  =  0.0
I 2 2 _ 1 3  =  0.0
I 2 2 _ 2 1  =  0.0
I 2 2 _ 2 2  =  0.0
I 2 2 _ 2 3  =  0.0
I 2 2 _ 3 1  =  0.0
I 2 2 _ 3 2  =  0.0
I 2 2 _ 3 3  =  0.0
I 2 3 _ 1 1  =  0.0
I 2 3 _ 1 2  =  0.0
I 2 3 _ 1 3  =  0.0
I 2 3 _ 2 1  =  0.0
I 2 3 _ 2 2  =  0.0


In [22]:
# Compute Dijkl (page 87 of Mura's book, equation 11.41)
def get_Dijkl(x,a,S):
    Dijkl = np.zeros((3,3,3,3))
    for i in range(3):
        for j in range(3):
            for k in range(3):
                for l in range(3):
                    t1 = 8*np.pi*(1-nu)*S[i,j,k,l] + 2*nu*delta[k,l]*x[i]*Ii_j(x,a,i,j)
                    t2 = (1+nu)*(delta[i,l]*x[k]*Ii_j(x,a,k,j) + delta[j,l]*x[k]*Ii_j(x,a,k,i) + delta[i,k]*x[l]*Ii_j(x,a,l,j) + delta[j,k]*x[l]*Ii_j(x,a,l,i))
                    t3 = -delta[i,j]*x[k]*(Ii_j(x,a,k,l) + a[i]**2*Iij_k(x,a,k,l,l)) - (delta[i,k]*x[j] + delta[j,k]*x[i])*(Ii_j(x,a,j,l) - a[i]**2*Iij_k(x,a,i,j,l))
                    t4 = -(delta[i,l]*x[j] + delta[j,l]*x[i])*(Ii_j(x,a,j,k) - a[i]**2*Iij_k(x,a,i,j,k)) - x[i]*x[j]*(Ii_jk(x,a,j,l,k) - a[i]**2*Iij_kl(x,a,i,j,l,k))
                    Dijkl[i,j,k,l] = (t1+t2+t3+t4)/(8*np.pi*(1-nu))
                    print("D",i+1,j+1,k+1,l+1," = ",Dijkl[i,j,k,l])
    return Dijkl

In [23]:
d = get_Dijkl(x,a,get_Sijkl(x,a))

S 1 1 1 1  =  0.31072416867053154
S 1 1 1 2  =  0.0
S 1 1 1 3  =  0.0
S 1 1 2 1  =  0.0
S 1 1 2 2  =  0.027375580674643397
S 1 1 2 3  =  0.0
S 1 1 3 1  =  0.0
S 1 1 3 2  =  0.0
S 1 1 3 3  =  -0.010448143082023012
S 1 2 1 1  =  0.0
S 1 2 1 2  =  0.1416757672271887
S 1 2 1 3  =  0.0
S 1 2 2 1  =  0.1416757672271887
S 1 2 2 2  =  0.0
S 1 2 2 3  =  0.0
S 1 2 3 1  =  0.0
S 1 2 3 2  =  0.0
S 1 2 3 3  =  0.0
S 1 3 1 1  =  0.0
S 1 3 1 2  =  0.0
S 1 3 1 3  =  0.20192788859690264
S 1 3 2 1  =  0.0
S 1 3 2 2  =  0.0
S 1 3 2 3  =  0.0
S 1 3 3 1  =  0.20192788859690264
S 1 3 3 2  =  0.0
S 1 3 3 3  =  0.0
S 2 1 1 1  =  0.0
S 2 1 1 2  =  0.14167575187117357
S 2 1 1 3  =  0.0
S 2 1 2 1  =  0.14167575187117357
S 2 1 2 2  =  0.0
S 2 1 2 3  =  0.0
S 2 1 3 1  =  0.0
S 2 1 3 2  =  0.0
S 2 1 3 3  =  0.0
S 2 2 1 1  =  0.02737978235299712
S 2 2 1 2  =  0.0
S 2 2 1 3  =  0.0
S 2 2 2 1  =  0.0
S 2 2 2 2  =  0.3107342324511871
S 2 2 2 3  =  0.0
S 2 2 3 1  =  0.0
S 2 2 3 2  =  0.0
S 2 2 3 3  =  -0.010448108248300

In [24]:
d == dummy

NameError: name 'dummy' is not defined

In [25]:
# Calculate epsilon tensor (page 85 of Mura's book, equation 11.33)
def get_epsilon(D,app_strain):
    epsilon = np.zeros((3,3))
    for i in range(3):
        for j in range(3):
            for k in range(3):
                for l in range(3):
                    epsilon[i,j] = epsilon[i,j] + D[i,j,k,l]*app_strain[k,l]
    return epsilon

In [26]:
# Calculate stress tensor for 3D
def get_stress(epsilon,E,nu,app_strain):
    epsilon = epsilon - app_strain
    sig11 = (E/((1+nu)*(1-2*nu)))*((1-nu)*epsilon[0,0] + nu*epsilon[1,1] + nu*epsilon[2,2])
    sig22 = (E/((1+nu)*(1-2*nu)))*(nu*epsilon[0,0] + (1-nu)*epsilon[1,1] + nu*epsilon[2,2])
    sig33 = (E/((1+nu)*(1-2*nu)))*(nu*epsilon[0,0] + nu*epsilon[1,1] + (1-nu)*epsilon[2,2])
    sig12 = (E/(1+nu))*epsilon[0,1]
    sig13 = (E/(1+nu))*epsilon[0,2]
    sig23 = (E/(1+nu))*epsilon[1,2]
    return np.array([[sig11,sig12,sig13],[sig12,sig22,sig23],[sig13,sig23,sig33]])

In [27]:
a1 = 24.001
a2 = 24
a3 = 12
a = np.array([a1, a2, a3])

eps11 = 0.001
eps22 = 0.001
eps33 = 0.001
eps12 = 0
eps13 = 0
eps23 = 0
app_strain = np.array([[eps11,eps12,eps13],[eps12,eps22,eps23],[eps13,eps23,eps33]])

nu = 0.3
E = 210


der_epsi = 0.001

delta = np.identity(3)
lambda_ = 1

lambda_ = get_lambda(x, a)
S = get_Sijkl(x, a)
D = get_Dijkl(x, a, S)
epsilon = get_epsilon(D, app_strain)
stress = get_stress(epsilon, E, nu, app_strain)
print(stress)

S 1 1 1 1  =  0.31072416867053154
S 1 1 1 2  =  0.0
S 1 1 1 3  =  0.0
S 1 1 2 1  =  0.0
S 1 1 2 2  =  0.027375580674643397
S 1 1 2 3  =  0.0
S 1 1 3 1  =  0.0
S 1 1 3 2  =  0.0
S 1 1 3 3  =  -0.010448143082023012
S 1 2 1 1  =  0.0
S 1 2 1 2  =  0.1416757672271887
S 1 2 1 3  =  0.0
S 1 2 2 1  =  0.1416757672271887
S 1 2 2 2  =  0.0
S 1 2 2 3  =  0.0
S 1 2 3 1  =  0.0
S 1 2 3 2  =  0.0
S 1 2 3 3  =  0.0
S 1 3 1 1  =  0.0
S 1 3 1 2  =  0.0
S 1 3 1 3  =  0.20192788859690264
S 1 3 2 1  =  0.0
S 1 3 2 2  =  0.0
S 1 3 2 3  =  0.0
S 1 3 3 1  =  0.20192788859690264
S 1 3 3 2  =  0.0
S 1 3 3 3  =  0.0
S 2 1 1 1  =  0.0
S 2 1 1 2  =  0.14167575187117357
S 2 1 1 3  =  0.0
S 2 1 2 1  =  0.14167575187117357
S 2 1 2 2  =  0.0
S 2 1 2 3  =  0.0
S 2 1 3 1  =  0.0
S 2 1 3 2  =  0.0
S 2 1 3 3  =  0.0
S 2 2 1 1  =  0.02737978235299712
S 2 2 1 2  =  0.0
S 2 2 1 3  =  0.0
S 2 2 2 1  =  0.0
S 2 2 2 2  =  0.3107342324511871
S 2 2 2 3  =  0.0
S 2 2 3 1  =  0.0
S 2 2 3 2  =  0.0
S 2 2 3 3  =  -0.010448108248300

In [ ]:
# stress_xx = []
# stress_yy = []
# stress_zz = []
# x_coord = np.linspace(1,2,2)
# for i in [0.1]:
#     x = np.array([i, 0, 0])
#     lambda_cubic(x, a)
#     S = get_Sijkl(x, a)
#     D = get_Dijkl(x, a, S)
#     epsilon = get_epsilon(D, app_strain)
#     stress = get_stress(epsilon, E, nu, app_strain)
#     print("Done for x = ",i)
#     stress_xx.append(stress[0,0])
#     stress_yy.append(stress[1,1])
#     stress_zz.append(stress[2,2])

# import matplotlib.pyplot as plt
# plt.plot(x_coord, stress_xx, label = "stress_xx")
# plt.plot(x_coord, stress_yy, label = "stress_yy")
# plt.plot(x_coord, stress_zz, label = "stress_zz")
# plt.xlabel('x')
# plt.ylabel('Stress')
# plt.legend()
# plt.show()

In [ ]:
# print("Stress_xx = ", stress_xx)
# print("Stress_yy = ", stress_yy)
# print("Stress_zz = ", stress_zz)